In [1]:
import sys
from pathlib import Path
import yaml
from tqdm import tqdm

import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torchdyn.core import NeuralODE
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.simplefilter("ignore", FutureWarning)

from field_model import *

In [2]:
# load config files for data and pipeline
with open("config.yaml", "r") as f1:
    config = yaml.full_load(f1)

with open("../../data/dataset_params.yaml") as f2:
    data_params = yaml.full_load(f2)

device = torch.device(config["device"])

# dir with trajectories Datasets
traj_dir = Path("trajectories/")
if not traj_dir.exists():
    raise FileNotFoundError("No Dataset created for given activity.")

# ode models
models_dir = Path("models/")

In [3]:
# compute mean and diag log std of the ML estimators
models = {}
for model_file in models_dir.glob("*"):
    activity = model_file.stem

    vector_field = VectorFieldMLP(
        config["trajectory_dim"], config["hidden_dim"])
    ode_model = NeuralODE(vector_field, solver='rk4').to(device)
    ode_model.load_state_dict(
        torch.load(model_file)
    )

    models[activity] = ode_model

Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.
Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.
Your vector field callable (nn.Module) should have both time `t` and state `x` as arguments, we've wrapped it for you.


In [4]:
def get_trajectory_mask(durations: torch.Tensor, traj: torch.Tensor) -> torch.Tensor:
    mask = torch.ones_like(traj).to(traj.device)
    for i in range(mask.shape[0]):
        # mask out padding vectors in trajectory
        mask[i, durations[i]: , ...] = 0.

    return mask

In [5]:
# classify test trajectories using ML principle
for model_file in models_dir.glob("*"):
    activity = model_file.stem

    print(f"Classifying {activity} label")

    test_loader = DataLoader(
        torch.load(traj_dir / f"{activity}_test.pt"),
        config["batch_size"],
        shuffle=False
    )
    model_loss= {}

    for model_name, ode_model in models.items():
        model_loss[model_name] = 0

        ode_model.eval()
        with torch.no_grad():
            for batch in tqdm(test_loader, desc=f"Testing {model_name} model", leave=True):
                traj: torch.Tensor = batch[0].to(device)
                durations: torch.Tensor = batch[1].to(device)

                traj_len = traj.shape[1]
                t_span = torch.arange(0, traj_len).to(device)
                mask = get_trajectory_mask(durations, traj)

                t_eval, traj_predict = ode_model(traj[:, 0, :], t_span)
                # move batch axis in front
                traj_predict = traj_predict.movedim(1, 0)

                # average loss among all real phase vectors
                loss = F.mse_loss(
                    traj.flatten(end_dim=-2),
                    (traj_predict * mask).flatten(end_dim=-2),
                    reduction="mean"
                ) * (traj.numel() / durations.sum())

                model_loss[model_name] += loss.item()

    print(model_loss)

    best_model_indx = np.array(list(model_loss.values())).argmin()
    best_model_name = list(model_loss.keys())[best_model_indx]

    print(f"Best model for true {activity} is {best_model_name} model.")

Classifying jog label


Testing std model: 100%|██████████| 10/10 [00:01<00:00,  6.85it/s]


{'jog': 2183.417251586914, 'wlk': 4718.659515380859, 'std': 5431.085754394531}
Best model for true jog is jog model.
Classifying wlk label


Testing std model: 100%|██████████| 21/21 [00:03<00:00,  6.27it/s]


{'jog': 4643.857955932617, 'wlk': 9451.685760498047, 'std': 11588.019104003906}
Best model for true wlk is jog model.
Classifying std label


Testing std model: 100%|██████████| 26/26 [00:03<00:00,  6.73it/s]

{'jog': 5119.088897705078, 'wlk': 10657.29898071289, 'std': 13499.962371826172}
Best model for true std is jog model.
